# Step 1. Representation image dataset and building CLIP-recommendations
Here we work with image dataset and create CLIP vectors

## 0. Preparation

Import necessary libraries

In [1]:
import torch
import os
import numpy as np
import random
import json
from transformers import CLIPImageProcessor, CLIPModel

Path of images forlder destination

In [2]:
dir_path = 'Dataset'

# 1. Representation of classes and functions

### 1.1. Dataset class: it is responsible for all operations with dataset

In [2]:
class Dataset:
    def __init__(self, img_dir_path):
        self.img_dir_path = img_dir_path

    def img_files(self, n=False):
        files = []
        for path in os.listdir(self.img_dir_path):
            if os.path.isfile(os.path.join(self.img_dir_path, path)):
                files.append(path)
        if n:
            random_files = random.sample(files, n)
            return random_files
        else:
            return files

### 1.2. CLIP class: it is responsible for building CLIP embeddings

In [18]:
#CLIP model representation
model_ID = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_ID)
preprocess = CLIPImageProcessor.from_pretrained(model_ID)

In [19]:
class CLIP:
    def __init__(self, image_list):
        self.image_list = image_list
    
    def get_features(self):
        n = len(self.image_list)
        for i in range(n):
            try:
                image = Image.open(f'{dir_path}/{self.image_list[i]}')
                image = preprocess(image, return_tensors="pt")
                image = image["pixel_values"]
                with torch.no_grad():
                    embedding = model.get_image_features(image)
                    embedding = embedding.reshape(embedding.shape[1]).tolist()
            except:
                print(f'{i} is failed')
            else:
                img_clip = {'img': self.image_list[i], 'clip': embedding}
                with open('clip.json', 'a') as clip:
                    json.dump(img_clip, clip)
                    clip.write('\n')

# 2. Building CLIP-vectors

In [10]:
#We form image array with all image files
dataset = Dataset(dir_path)
img_list = dataset.img_files()

In [74]:
#We build CLIP vectors using obtained image array
clip = CLIP(img_list)
features = clip.get_features()